
**Install requirements**

In [1]:
!pip3 install 'torch==1.3.1'
!pip3 install 'torchvision==0.5.0'
!pip3 install 'Pillow-SIMD'
!pip3 install 'tqdm'
!pip3 install 'matplotlib'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement torch==1.3.1 (from versions: 1.4.0, 1.5.0, 1.5.1, 1.6.0, 1.7.0, 1.7.1, 1.8.0, 1.8.1, 1.9.0, 1.9.1, 1.10.0, 1.10.1, 1.10.2, 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1)
ERROR: No matching distribution found for torch==1.3.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 753.4/753.4 MB 1.9 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.0+cu116
    Uninstalling torch-1.13.0+cu116:
      Successfully uninstalled torch-1.13.0+cu116
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.14.0+cu116
    Uninstalling torchvision-0.14.0+cu116:
      Successfully uninstalled torchvision-0.14.0+cu116

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Import libraries**

In [11]:
import os
import logging

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn

import torchvision
from torchvision import transforms
from torchvision.models import alexnet


from PIL import Image
from tqdm import tqdm

**Set Arguments**

In [12]:
DEVICE = 'cuda' # 'cuda' or 'cpu'

NUM_CLASSES = 100

BATCH_SIZE = 128     # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                     # the batch size, learning rate should change by the same factor to have comparable results

LR = 1e-1            # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 1e-4  # Regularization, you can keep this at the default

NUM_EPOCHS = 160     # Total number of training epochs (iterations over dataset)
STEP_SIZE = 10       # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1          # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = 1000

**Define Data Preprocessing**

In [13]:
# This dataset has 100 classes containing 600 images each. There are 500 training images and 100 testing images per 
# class. The 100 classes in the CIFAR-100 are grouped into 20 superclasses. Each image comes with a "fine" label (the class to which it belongs) and a 
# "coarse" label (the superclass to which it belongs).
# Define transforms for training phase

# random crop, random horizontal flip, per-pixel normalization 

train_transform = transforms.Compose([transforms.Resize(48),      # Resizes short size of the PIL image to 256
                                      transforms.RandomCrop(32),  # Crops a random square patch of the image for a 224x224 input
                                                                   # Remember this when applying different transformations, otherwise you get an error
                                      transforms.RandomHorizontalFlip(0.5), # 0.5 probability
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406], # TODO: Check 
                                                           std=[0.229, 0.224, 0.225])
                                      #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalizes tensor with mean and standard deviation
])
# Define transforms for the evaluation phase
eval_transform = transforms.Compose([transforms.Resize(48),
                                      transforms.RandomCrop(32),
                                      transforms.RandomHorizontalFlip(0.5), 
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                           std=[0.229, 0.224, 0.225])
                                      #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalizes tensor with mean and standard deviation
])

**Prepare Dataset**

In [14]:
# Clone github repository with data
if not os.path.isdir('./CentralizedResnet'):
  !git clone https://github.com/AML-SergioMejia/CentralizedResNet.git

root_dir = "CIFAR100"
""""
root: str,
train: bool = True, 
transform: Optional[Callable] = None, 
target_transform: Optional[Callable] = None, 
download: bool = False
"""
# torchvision.datasets.CIFAR100(root: str, train: bool = True, transform: Optional[Callable] = None, target_transform: Optional[Callable] = None, download: bool = False)

# Prepare Pytorch train/test Datasets
train_dataset = torchvision.datasets.CIFAR100( root_dir, transform=train_transform, download=True)#Caltech(DATA_DIR, split='train',  transform=train_transform)
test_dataset = torchvision.datasets.CIFAR100( root_dir, train=False, transform=train_transform, download=True)
#print('Initial Train Dataset: {}'.format(len(train_dataset)))

#stratified_sampling = train_dataset.stratified_sample(2)
#train_indexes = stratified_sampling[0] # split the indices for your train split
#val_indexes = stratified_sampling[1] # split the indices for your val split

#val_dataset = Subset(train_dataset, val_indexes)
#train_dataset = Subset(train_dataset, train_indexes)
#NUM_CLASSES -= 1 # We remove the BACKGROUND class

# Check dataset sizes
print('Train Dataset: {}'.format(len(train_dataset)))
# print('Valid Dataset: {}'.format(len(val_dataset)))
print('Test Dataset: {}'.format(len(test_dataset)))

fatal: destination path 'CentralizedResNet' already exists and is not an empty directory.
Files already downloaded and verified
Files already downloaded and verified
Train Dataset: 50000
Test Dataset: 10000


**Prepare Dataloaders**

In [15]:
# Dataloaders iterate over pytorch datasets and transparently provide useful functions (e.g. parallelization and shuffling)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
# val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

**Prepare Network**

In [16]:
from CentralizedResNet.resnet_cifar import ResNet20
net = ResNet20(num_blocks=3, num_classes=100)
# We need 100 outputs
# net.classifier[6] = nn.Linear(4096, NUM_CLASSES) # nn.Linear in pytorch is a fully connected layer
#                                                  # The convolutional layer is nn.Conv2d

# We just changed the last layer with a new fully connected layer with 100 outputs

**Prepare Training**

In [17]:
# Define loss function
criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy

# Choose parameters to optimize
# To access a different set of parameters, you have to access submodules of AlexNet
# (nn.Module objects, like AlexNet, implement the Composite Pattern)
# e.g.: parameters of the fully connected layers: net.classifier.parameters()
# e.g.: parameters of the convolutional layers: look at alexnet's source code ;) 
parameters_to_optimize = net.parameters() # In this case we optimize over all the parameters of AlexNet

# Define optimizer
# An optimizer updates the weights based on loss
# We use SGD with momentum
optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

# Define scheduler
# A scheduler dynamically changes learning rate
# The most common schedule is the step(-down), which multiplies learning rate by gamma every STEP_SIZE epochs
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)

**Train**

In [ ]:
# By default, everything is loaded to cpu
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda

cudnn.benchmark # Calling this optimizes runtime

current_step = 0
# Start iterating over the epochs
for epoch in range(NUM_EPOCHS):
  print('Starting epoch {}/{}'.format(epoch+1, NUM_EPOCHS))#, scheduler.get_lr()))

  i = 0
  # Iterate over the dataset
  for images, labels in train_dataloader:
    # Bring data over the device of choice
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    net.train() # Sets module in training mode

    # PyTorch, by default, accumulates gradients after each backward pass
    # We need to manually set the gradients to zero before starting a new iteration
    optimizer.zero_grad() # Zero-ing the gradients

    # Forward pass to the network
    outputs = net(images)

    # Compute loss based on output and ground truth
    loss = criterion(outputs, labels)

    # Log loss
    if current_step % LOG_FREQUENCY == 0:
      print('Step {}, Loss {}'.format(current_step, loss.item()))

    # Compute gradients for each layer and update weights
    loss.backward()  # backward pass: computes gradients
    optimizer.step() # update weights based on accumulated gradients
    scheduler.step(epoch + i / len(train_dataloader))
    i += 1

    current_step += 1
  # end for dataset iteration

  # Step the scheduler
  # scheduler.step() 

Starting epoch 1/160
Step 0, Loss 4.60814905166626
Starting epoch 2/160
Starting epoch 3/160
Step 1000, Loss 4.557133674621582
Starting epoch 4/160
Starting epoch 5/160
Starting epoch 6/160
Step 2000, Loss 4.558177947998047
Starting epoch 7/160
Starting epoch 8/160
Step 3000, Loss 4.548575401306152
Starting epoch 9/160
Starting epoch 10/160
Starting epoch 11/160
Step 4000, Loss 4.521381378173828
Starting epoch 12/160
Starting epoch 13/160
Step 5000, Loss 4.554089069366455
Starting epoch 14/160
Starting epoch 15/160
Starting epoch 16/160
Step 6000, Loss 4.528486251831055
Starting epoch 17/160
Starting epoch 18/160
Step 7000, Loss 4.525395393371582
Starting epoch 19/160
Starting epoch 20/160
Starting epoch 21/160
Step 8000, Loss 4.555215835571289
Starting epoch 22/160
Starting epoch 23/160
Starting epoch 24/160
Step 9000, Loss 4.498642444610596
Starting epoch 25/160
Starting epoch 26/160
Step 10000, Loss 4.529279708862305


**Validation on last state of network**

In [ ]:
def validate_input(network, dataset):
  network = network.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
  network.train(False) # Set Network to evaluation mode
  val_dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

  running_corrects = 0
  running_loss = 0.0
  for input, labels in tqdm(val_dataloader):
    input = input.to(DEVICE)
    labels = labels.to(DEVICE)

    # Forward Pass
    outputs = network(input)

    loss = criterion(outputs, labels)
    
    running_loss += loss.item()

    # Get predictions
    _, preds = torch.max(outputs.data, 1)

    # Update Corrects
    running_corrects += torch.sum(preds == labels.data).data.item()

  # Calculate Accuracy
  accuracy = running_corrects / float(len(dataset))
  loss = running_loss / len(dataset)
  return accuracy, loss
accuracy = validate_input(net, val_dataset)
print('Validation Accuracy: {}'.format(accuracy))

**Training + validation per epoch**

In [ ]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# !! Call "Prepare training" cell again before executing this module !!
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# By default, everything is loaded to cpu
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda

cudnn.benchmark # Calling this optimizes runtime

current_step = 0

loss_values = []
acc_values = []

# Start iterating over the epochs
for epoch in range(NUM_EPOCHS):
  print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, scheduler.get_lr()))

  running_loss = 0.0
  running_correct = 0
  # Iterate over the dataset
  for images, labels in train_dataloader:
    # Bring data over the device of choice
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    net.train() # Sets module in training mode

    # PyTorch, by default, accumulates gradients after each backward pass
    # We need to manually set the gradients to zero before starting a new iteration
    optimizer.zero_grad() # Zero-ing the gradients

    # Forward pass to the network
    outputs = net(images)

    # Compute loss based on output and ground truth
    loss = criterion(outputs, labels)
    
    running_loss += loss.item()
    _, preds = torch.max(outputs.data, 1)
    running_correct += torch.sum(preds == labels.data).data.item()

    # Log loss
    if current_step % LOG_FREQUENCY == 0:
      print('Step {}, Loss {}'.format(current_step, loss.item()))

    # Compute gradients for each layer and update weights
    loss.backward()  # backward pass: computes gradients
    optimizer.step() # update weights based on accumulated gradients

    current_step += 1
  # end for dataset iteration
  
  #outputs = net(val_dataset.)
  #loss_val = criterion(outputs, val_dataset.dataset.labels)
  accuracy_val, loss_val = validate_input(net, val_dataset)
  
  #outputs = net(train_dataset)
  #loss_train = criterion(outputs, train_dataset.dataset.labels)
  loss_train = running_loss/len(train_dataloader.dataset)
  accuracy_train = running_correct/len(train_dataloader.dataset)
  
  loss_values.append([loss_train, loss_val])
  acc_values.append([accuracy_train, accuracy_val])
  print('Validation Accuracy: {}'.format(accuracy_val))

  # Step the scheduler
  scheduler.step() 

In [ ]:
from matplotlib import pyplot as plt

plt.plot(range(NUM_EPOCHS), loss_values)
plt.legend(["Train", "Val"])
plt.xlabel("Epochs")
plt.show()

plt.plot(range(NUM_EPOCHS), acc_values)
plt.legend(["Train", "Val"])
plt.xlabel("Epochs")
plt.show()


**Test**

In [ ]:
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
net.train(False) # Set Network to evaluation mode

running_corrects = 0
for images, labels in tqdm(test_dataloader):
  images = images.to(DEVICE)
  labels = labels.to(DEVICE)

  # Forward Pass
  outputs = net(images)

  # Get predictions
  _, preds = torch.max(outputs.data, 1)

  # Update Corrects
  running_corrects += torch.sum(preds == labels.data).data.item()

# Calculate Accuracy
accuracy = running_corrects / float(len(test_dataset))

print('Test Accuracy: {}'.format(accuracy))

100%|██████████| 12/12 [00:06<00:00,  1.80it/s]

Test Accuracy: 0.840995506394746
